In [11]:
from cmapPy.pandasGEXpress.parse_gct import parse
from cmapPy.pandasGEXpress.write_gct import write

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

data = parse('GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct', ridx=[i for i in range(600)])
data.row_metadata_df.dropna(inplace=True)
data.data_df = data.data_df.loc[data.row_metadata_df.index]
dataset = data.data_df

print(len(dataset))

In [ ]:
#To remove all entries for whom all values are 0, to prevent corr and p being NaN


In [33]:
cols = [ i for i in range(100)]
print(usecols)
dataset1 = pd.read_csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', index_col = 'Name', usecols = cols, sep = '\t', header = 2 , nrows = 100)
dataset = dataset1.drop('Description', axis = 1)
print(len(dataset))

for i in dataset.index.values:
    if np.sum(dataset.loc[i].values) == 0.0:
        dataset = dataset.drop(i, axis=0)
            
print(len(dataset))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
100
97


In [34]:
individuals = dataset.index.values
bootstrapped_individuals = np.random.choice(individuals, len(individuals), replace=True)

In [35]:
p_values = []
corr_values = []

count = 0
for i in bootstrapped_individuals:
    for j in bootstrapped_individuals:
        
        count = count + 1
        if(count%100 == 0):
            print(count)

        gene1 = dataset.loc[i].values
        gene2 = dataset.loc[j].values
        
        corr, p = spearmanr(gene1, gene2)
        
        corr_values.append(corr)
        p_values.append(p)

print(p_values)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
[0.0, 0.4025601457305763, 0.2255562276136511, 0.7488064870917306, 0.1095894013514051, 0.019954292656087926, 0.7019958187676887, 0.9674968684393768, 0.25927178332367534, 0.5820138233885821, 0.13088709742217444, 0.542893643099266, 0.2881785764587561, 0.5820138233885821, 0.6230474161542611, 0.9674968684393768, 9.110036283895282e-09, 0.36916810582018855, 0.3160299899943403, 0.01963808597685102, 0.0006097417521284559, 0.9223470638625377, 0.0953666158844479, 0.10228731369363023, 0.9674968684393768, 0.7431399099277796, 0.029660902940750276,

In [26]:
results = multipletests(p_values, alpha = 0.05, method='fdr_bh')
reject = results[0]
print(len(reject))
p_values_corrected = results[1]
results

9604


(array([ True,  True,  True, ...,  True,  True,  True]),
 array([0.00000000e+000, 4.68137438e-002, 2.31944259e-045, ...,
        1.27604906e-278, 8.90810373e-124, 0.00000000e+000]),
 5.34081189229596e-06,
 5.206164098292378e-06)

In [36]:
#Making the Graph. It is a dictionary of the form {gene : [array of genes to which it has an edge]}
G = {}

for i in range(len(dataset)):
    temp = []
    for j in range(len(dataset)):
        if p_values_corrected[i*len(dataset) + j] > 0.1 and not reject[i*len(dataset) + j]:  #If its greater than cutoff and not rejected during multiple test correction
            temp.append(dataset.index[j])
            
    G.update({dataset.index[i] : temp})
    
G

{'ENSG00000223972.4': ['ENSG00000241860.2',
  'ENSG00000269732.1',
  'ENSG00000228327.2',
  'ENSG00000237330.2',
  'ENSG00000186891.9',
  'ENSG00000127054.14',
  'ENSG00000240731.1'],
 'ENSG00000227232.4': ['ENSG00000223972.4',
  'ENSG00000268020.2',
  'ENSG00000240361.1',
  'ENSG00000233750.3',
  'ENSG00000237094.7',
  'ENSG00000198744.5',
  'ENSG00000235373.1',
  'ENSG00000229905.1',
  'ENSG00000230092.3',
  'ENSG00000269831.1',
  'ENSG00000228794.4',
  'ENSG00000224969.1',
  'ENSG00000237330.2',
  'ENSG00000207607.1',
  'ENSG00000272141.1',
  'ENSG00000205231.1',
  'ENSG00000176022.3',
  'ENSG00000127054.14'],
 'ENSG00000243485.2': ['ENSG00000185097.2',
  'ENSG00000228794.4',
  'ENSG00000230415.1'],
 'ENSG00000237613.2': ['ENSG00000268020.2',
  'ENSG00000237683.5',
  'ENSG00000241860.2',
  'ENSG00000269308.1',
  'ENSG00000176022.3',
  'ENSG00000107404.13'],
 'ENSG00000268020.2': ['ENSG00000240361.1',
  'ENSG00000268903.1',
  'ENSG00000222623.1',
  'ENSG00000228463.4',
  'ENSG0000023

In [30]:
import pickle
f = open("file2.pkl","wb")

pickle.dump(G,f)
f.close()
print("Graph saved")

Graph saved


In [31]:
len(G)

510